In [ ]:
import os
from azureml.core import Workspace, Datastore, Dataset
from azureml.pipeline.wrapper import Module, dsl, Pipeline
from azureml.pipeline.wrapper._dataset import get_global_dataset_by_path
from azureml.core.compute import AmlCompute, ComputeTarget

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

aml_compute_target = "aml-compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except:
    print("Creating new compute target: {}".format(aml_compute_target))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1,
                                                                max_nodes=4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

In [ ]:
blob_input_data = get_global_dataset_by_path(
    ws, 'Automobile_price_data',
    'GenericCSV/Automobile_price_data_(Raw)')
notebook_module_func = Module.from_notebook(ws, 'authoring/sample_module_from_notebook/from_notebook.ipynb')
module1 = notebook_module_func(input=blob_input_data, param1='aaa0', param2='bbb0',i=1,j=2,flag=False)
module2 = notebook_module_func(input=module1.outputs['output'], param1='aaa1', param2='bbb1',i=2,j=3,flag=True)
steps = [module1, module2]

In [ ]:
pipeline_name = "notebook_test"
pipeline = Pipeline(name=pipeline_name, nodes=steps,
                    outputs={**steps[-1].outputs})
run = pipeline.submit(pipeline_name, "aml-compute")
print(run.id)
run.wait_for_completion()



In [ ]:
pipeline.run()

You can also create module from notebook file by using Azure CLI

In [ ]:
!az ml module init --source authoring/sample_module_from_notebook/from_notebook.ipynb